In [1]:
import sqlite3
import pandas as pd

In [11]:
covid_19_indonesia = pd.read_csv("covid_19_indonesia_time_series_all.csv")
covid_19_indonesia.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,39,20,41,...,-6.204699,0.18,3.60,0.0,1.84,0.18,51.28%,105.13%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,41,20,41,...,-6.204699,0.18,3.78,0.0,1.84,0.18,48.78%,100.00%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,-0.789275,0.01,0.01,0.0,0.00,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-RI,Riau,1,0,0,1,2,0,3,...,0.511648,0.16,0.33,0.0,0.00,0.00,0.00%,150.00%,NaN,NaN
4,3/3/2020,ID-JK,DKI Jakarta,2,0,0,2,43,20,41,...,-6.204699,0.18,3.96,0.0,1.84,0.18,46.51%,95.35%,1.0,1.0


In [3]:
covid_19_indonesia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21759 entries, 0 to 21758
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         21759 non-null  object 
 1   Location ISO Code            21759 non-null  object 
 2   Location                     21759 non-null  object 
 3   New Cases                    21759 non-null  int64  
 4   New Deaths                   21759 non-null  int64  
 5   New Recovered                21759 non-null  int64  
 6   New Active Cases             21759 non-null  int64  
 7   Total Cases                  21759 non-null  int64  
 8   Total Deaths                 21759 non-null  int64  
 9   Total Recovered              21759 non-null  int64  
 10  Total Active Cases           21759 non-null  int64  
 11  Location Level               21759 non-null  object 
 12  City or Regency              0 non-null      float64
 13  Province        

In [4]:
# Membuat fungsi koneksi query SQLite

def create_connection(path):
    connection=sqlite3.connect(path)
    return(connection)

connection = create_connection("statdescriptive.sqlite")

In [5]:
# Membuat fungsi eksekusi query SQLite

def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    connection.commit()

In [6]:
# Membuat tabel data Covid-19 di Indonesia

table_1 = ''' 
CREATE TABLE IF NOT EXISTS total_covid_19 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Date OBJECT,
    Location OBJECT,
    Total_Cases INTEGER,
    Total_Deaths INTEGER,
    Total_Recovered INTEGER,
    Total_Active_Cases INTEGER
)
'''

execute_query(connection, table_1)

In [7]:
for i in range(len(covid_19_indonesia)):
    dat = covid_19_indonesia.values[i,:]
    insert_values = f'''
    INSERT INTO total_covid_19 (
        Date, Location, Total_Cases, Total_Deaths, Total_Recovered, Total_Active_Cases)
    VALUES ("{dat[0]}", "{dat[2]}", {dat[7]}, {dat[8]}, {dat[9]}, {dat[10]})
    '''
    execute_query(connection, insert_values)

In [8]:
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_sequence;")
print(cursor.fetchall())

[('total_covid_19',)]


In [12]:
covid_19_query = pd.read_sql_query('select * from total_covid_19',connection)
covid_19_query.head()

,id,Date,Location,Total_Cases,Total_Deaths,Total_Recovered,Total_Active_Cases
0,1,3/1/2020,DKI Jakarta,39,20,41,-22
1,2,3/2/2020,DKI Jakarta,41,20,41,-20
2,3,3/2/2020,Indonesia,2,0,0,2
3,4,3/2/2020,Riau,2,0,3,-1
4,5,3/3/2020,DKI Jakarta,43,20,41,-18


In [13]:
covid_19_query.describe().T

,count,mean,std,min,25%,50%,75%,max
id,21759.0,10880.000000,6281.426590,1.0,5440.5,10880.0,16319.5,21759.0
Total_Cases,21759.0,85259.970817,368513.285849,1.0,1822.5,10780.0,36464.5,4257243.0
Total_Deaths,21759.0,2648.289352,11776.011067,0.0,50.0,283.0,1050.0,143858.0
Total_Recovered,21759.0,76712.602463,340395.710890,0.0,1038.5,8745.0,32932.5,4105680.0
Total_Active_Cases,21759.0,5899.079002,27518.102431,-2306.0,182.0,919.0,2607.5,574135.0


In [17]:
print("Rata-rata total kasus Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Cases.mean())
print("Rata-rata total kematian akibat Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Deaths.mean())
print("Rata-rata total kesembuhan Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Recovered.mean())
print("Rata-rata total kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia:", covid_19_query.Total_Active_Cases.mean())

Rata-rata total kasus Covid-19 di setiap wilayah Indonesia: 85259.97081667357
Rata-rata total kematian akibat Covid-19 di setiap wilayah Indonesia: 2648.289351532699
Rata-rata total kesembuhan Covid-19 di setiap wilayah Indonesia: 76712.6024633485
Rata-rata total kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia: 5899.079001792362


In [18]:
print("Nilai tengah kasus Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Cases.median())
print("Nilai tengah kematian akibat Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Deaths.median())
print("Nilai tengah kesembuhan Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Recovered.median())
print("Nilai tengah kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia:", covid_19_query.Total_Active_Cases.median())

Nilai tengah kasus Covid-19 di setiap wilayah Indonesia: 10780.0
Nilai tengah kematian akibat Covid-19 di setiap wilayah Indonesia: 283.0
Nilai tengah kesembuhan Covid-19 di setiap wilayah Indonesia: 8745.0
Nilai tengah kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia: 919.0


In [19]:
print("Nilai mode kasus Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Cases.mode())
print("Nilai mode kematian akibat Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Deaths.mode())
print("Nilai mode kesembuhan Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Recovered.mode())
print("Nilai mode kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia:", covid_19_query.Total_Active_Cases.mode())

Nilai mode kasus Covid-19 di setiap wilayah Indonesia: 0    2
Name: Total_Cases, dtype: int64
Nilai mode kematian akibat Covid-19 di setiap wilayah Indonesia: 0    1
Name: Total_Deaths, dtype: int64
Nilai mode kesembuhan Covid-19 di setiap wilayah Indonesia: 0    0
Name: Total_Recovered, dtype: int64
Nilai mode kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia: 0    1
Name: Total_Active_Cases, dtype: int64


In [20]:
print("Nilai variasi kasus Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Cases.var())
print("Nilai variasi kematian akibat Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Deaths.var())
print("Nilai variasi kesembuhan Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Recovered.var())
print("Nilai variasi kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia:", covid_19_query.Total_Active_Cases.var())

Nilai variasi kasus Covid-19 di setiap wilayah Indonesia: 135802041847.52693
Nilai variasi kematian akibat Covid-19 di setiap wilayah Indonesia: 138674436.65034586
Nilai variasi kesembuhan Covid-19 di setiap wilayah Indonesia: 115869239992.45517
Nilai variasi kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia: 757245961.4085474


In [21]:
print("Nilai deviasi standar kasus Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Cases.std())
print("Nilai deviasi standar kematian akibat Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Deaths.std())
print("Nilai deviasi standar kesembuhan Covid-19 di setiap wilayah Indonesia:", covid_19_query.Total_Recovered.std())
print("Nilai deviasi standar kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia:", covid_19_query.Total_Active_Cases.std())

Nilai deviasi standar kasus Covid-19 di setiap wilayah Indonesia: 368513.28584940726
Nilai deviasi standar kematian akibat Covid-19 di setiap wilayah Indonesia: 11776.011067010164
Nilai deviasi standar kesembuhan Covid-19 di setiap wilayah Indonesia: 340395.7108902155
Nilai deviasi standar kasus Covid-19 yang masih berjalan di setiap wilayah Indonesia: 27518.102431100648
